In [2]:
import os
import modeling.erp as erp
import numpy as np
import pandas as pd

base_path = f"data/outputdata/dimensions/"
df_products    = pd.read_csv(os.path.join(base_path, "product.csv"))
df_services    = pd.read_csv(os.path.join(base_path, "service.csv"))
df_procurement = pd.read_csv(os.path.join(base_path, "procurement.csv"))
df_departments = pd.read_csv(os.path.join(base_path, "department.csv"))
df_accounts    = pd.read_csv("data/inputdata/coa_general.csv", sep=";")
df_customers   = pd.read_csv(os.path.join(base_path, "customer.csv"))
df_vendors     = pd.read_csv(os.path.join(base_path, "vendor.csv"))

In [3]:
estimated_financials = 1000000


    # Apply proportional spend estimates
df_procurement["annual_spend"] = np.round(
    estimated_financials * df_procurement["proportionality"], -3
)
df_products["annual_spend"] = np.round(
    estimated_financials * df_products["proportionality"], -3
)
df_services["annual_spend"] = np.round(
    estimated_financials * df_services["proportionality"], -3
)
df_departments["annual_spend"] = np.round(
    estimated_financials * df_departments["proportionality"], -3
)



In [28]:
from generators.full_generators import create_all_erp_data

generated_data = {
        "df_erp_expenses": df_erp_expenses,
        "df_map_expenses": df_map_expenses,
        "df_erp_products": df_erp_products,
        "df_map_products": df_map_products,
    }
data = create_all_erp_data(generated_data, company_name="Semler")

✔ All erp-data generated.
✔ All ERP CSVs saved to: data/outputdata/fact


In [29]:
erp_all = data["df_erp_all"]

In [36]:
erp_all.sort_values("unit_price", ascending=True).head(20)

,document_number,debit_credit,date,amount,quantity,unit_price,account_id,product_id,procurement_id,service_id,vendor_id,customer_id
2457,78001178,Credit,2024-03-25,4.0,2,2.0,Office Furniture & Equipment,None,Office push pins,None,OfficeTeam Danmark,NaN
2458,78002725,Credit,2024-03-27,2.0,1,2.0,Office Furniture & Equipment,None,Office push pins,None,OfficeTeam Danmark,NaN
2459,78001162,Credit,2024-03-01,8.0,4,2.0,Office Furniture & Equipment,None,Office push pins,None,OfficeTeam Danmark,NaN
2460,78001592,Credit,2024-03-28,50.0,25,2.0,Office Furniture & Equipment,None,Office push pins,None,OfficeTeam Danmark,NaN
2427,78002173,Credit,2024-01-11,200.0,100,2.0,Office Furniture & Equipment,None,Office paper clips,None,OfficeTeam Danmark,NaN
2428,78002512,Credit,2024-02-10,28.0,14,2.0,Office Furniture & Equipment,None,Office paper clips,None,OfficeTeam Danmark,NaN
2429,78001145,Credit,2024-02-01,0.0,1,2.0,Office Furniture & Equipment,None,Office paper clips,None,OfficeTeam Danmark,NaN
2430,78001858,Credit,2024-03-07,10.0,5,2.0,Office Furniture & Equipment,None,Office paper clips,None,OfficeTeam Danmark,NaN
2431,78001977,Credit,2024-03-27,2.0,1,2.0,Office Furniture & Equipment,None,Office paper clips,None,OfficeTeam Danmark,NaN
2432,78002375,Credit,2024-03-05,12.0,6,2.0,Office Furniture & Equipment,None,Office paper clips,None,OfficeTeam Danmark,NaN


In [1]:
import pandas as pd 
import utils.prompt_utils as prompt_utils
import utils.utils as utils
import generators.random_generators as random_generators
import numpy as np
import re

import random
import numpy as np
import pandas as pd
import os
from utils.utils_llm import create_mapping_from_metadata
from typing import Dict, Any, List, Optional
import time
import random

import openai

import dotenv

import utils.prompt_utils as prompt_utils
import generators.random_generators as random_generators
from concurrent.futures import ThreadPoolExecutor, as_completed
import re, hashlib, json, dotenv
from openai import OpenAI


In [2]:
def generate_business_units_llm(
    company_name: str,
    count: int = 9,
    model: str = "gpt-4.1",
    temp: float = 0.8,
) -> str:
    """
    LLM-driven generator for Business Units (BUs) inside a single company.
    Returns a CSV string with columns: bu_key,bu_name,bu_type,region,currency,fiscal_start_month,sells_to,buys_from,notes

    - bu_key: UPPER_SNAKE, prefixed with company acronym (e.g., ACME_SALES)
    - bu_type: one of [Manufacturing, Sales, Services, SharedServices, RnD, Logistics, ECommerce, Retail, Consulting]
    - region: optional (e.g., Denmark, Nordics, EU North) — keep Denmark-centric / realistic
    - sells_to / buys_from: semicolon-separated list of other bu_keys (optional; use for intercompany)
    - currency: ISO (DKK/EUR/GBP, etc.)
    - fiscal_start_month: 1–12 (integers)
    - notes: 1 short sentence about what makes the BU “work differently” (patterns, seasonality, pricing, etc.)

    Distribution guidance (soft constraints):
      - Ensure coverage of core functions:
        * ≥1 Sales-type BU (Sales, Retail, ECommerce)
        * ≥1 Delivery/Production BU (Manufacturing, Logistics, Services)
        * ≥1 Shared function (SharedServices or similar)
      - Max 1–2 regionally scoped BUs (e.g., “Nordics Sales”)
      - Keep Denmark-only operations in mind (no “Global Country Manager” roles)

    Output is intentionally over-generated (x1.4) and later truncated to 'count'.
    """

    client = prompt_utils.get_openai_client()

    over_request_count = int(np.floor(int(count) * 1.4))
    header = "companyname;companycode;bu_key;bu_name;bu_type;sells_to;buys_from"
    constraints = prompt_utils.get_standard_constraints(header, over_request_count)
    ctxb = prompt_utils._ctx_block(company_name)

    # Build a stable company prefix for keys (ACME, LEGO, etc.)
    company_prefix = re.sub(r"[^A-Za-z0-9]+", "", company_name).upper()
    company_prefix = company_prefix[:8] if company_prefix else "COMPANY"

    prompt = f"""
    You are an experienced CFO/COO designing Business Units (BUs) for a company named {company_name}.
    Generate {over_request_count} BUs and companies as a CSV with the columns:
    {header}

    Rules & style:
    - companyname: create {np.floor(over_request_count/3)} geographical companies (e.g. {company_name}_denmark, {company_name}_sweden).
    - companycode: start at 1000 and increment for each new company.
    - bu_key: UPPER_SNAKE, prefixed with {company_prefix}_ (e.g., {company_prefix}_ELECTRICITY).  
      Each company must have ~3 BUs. Use industry-specific terms relevant to {company_name}, 
      not generic "Sales" or "Manufacturing".
    - bu_name: human-readable, e.g., "Electricity Generation", "District Heating", "Biogas Production", "Carbon Consulting".
    - bu_type: choose from [Manufacturing, Services, SharedServices, RnD, Retail, Consulting].
      Map your industry-specific BU into the closest category.
    - sells_to / buys_from: list of other bu_keys (ONLY THE ONES THAT HAVE BEEN GENERATED) if intercompany flows apply; allow empty. Only one intercompany per BU.

    {constraints}
    """.strip()

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful finance data modeller and HR/operations expert."},
            {"role": "user", "content": prompt},
        ],
        temperature=temp,
    )

    # Reuse your existing CSV parser/truncator
    return prompt_utils.parse_and_truncate_csv(response.choices[0].message.content, count)

In [3]:
df_products = pd.read_csv("data/outputdata/dimensions/product.csv")
df_procurement = pd.read_csv("data/outputdata/dimensions/procurement.csv")
df_services = pd.read_csv("data/outputdata/dimensions/service.csv")
df_departments = pd.read_csv("data/outputdata/dimensions/department.csv")
df_accounts = pd.read_csv("data/inputdata/coa_general.csv", sep=";")
df_customers = pd.read_csv("data/outputdata/dimensions/customer.csv")
df_vendors = pd.read_csv("data/outputdata/dimensions/vendor.csv")
df_bu_companies = generate_business_units_llm(company_name="BioCirc", count=9)

In [7]:
import random
import numpy as np
import pandas as pd
import os
from utils.utils_llm import create_mapping_from_metadata
from typing import Dict, Any, List, Optional
import time
import random

import openai

import dotenv

import utils.prompt_utils as prompt_utils
import generators.random_generators as random_generators


from concurrent.futures import ThreadPoolExecutor, as_completed
import re, hashlib, json, dotenv
from openai import OpenAI

def _overlap_tokens(s: str) -> set:
    return set(re.findall(r"[a-z0-9]+", str(s).lower()))

def _shortlist_accounts(df_accounts, allowed, item_name, k=80):
    toks = _overlap_tokens(item_name)
    sub = df_accounts[df_accounts["GLLevel02"] == allowed][["AccountKey","name","GLLevel02"]].dropna().copy()
    sub["AccountKey"] = sub["AccountKey"].astype(str)
    sub["__score"] = sub["name"].astype(str).apply(lambda s: len(toks & _overlap_tokens(s)))
    sub = sub.sort_values("__score", ascending=False).head(k).drop(columns="__score")

    records = []
    for i, r in sub.reset_index(drop=True).iterrows():
        records.append({
            "idx": i,
            "AccountKey": r["AccountKey"],
            "name": str(r["name"]),             # ← use the column, not r.name
            "GLLevel02": r["GLLevel02"],
        })
    return records

def _shortlist_partners(df_partners, item_name, account_name, k=200):
    if df_partners is None or "name" not in df_partners.columns or df_partners.empty:
        return []
    toks = _overlap_tokens(item_name) | _overlap_tokens(account_name)
    sub = df_partners.copy()
    sub["__score"] = sub["name"].astype(str).apply(lambda s: len(toks & _overlap_tokens(s)))
    sub = sub.sort_values("__score", ascending=False).head(k).drop(columns="__score")
    return [{"idx":i,"name":n} for i,n in enumerate(sub["name"].astype(str).tolist())]

def _det_pick_idx(n: int, key: str) -> int:
    h = hashlib.sha1(key.encode("utf-8")).hexdigest()
    return int(h, 16) % max(1, n)

def _pick_idx_with_gpt(client, system_schema_text, payload, n, fallback_key, model="gpt-4.1", retries=2):
    backoff = 0.6
    for attempt in range(retries + 1):
        try:
            resp = client.chat.completions.create(
                model=model,
                response_format={"type":"json_object"},
                messages=[
                    {"role":"system","content":system_schema_text},
                    {"role":"user","content":json.dumps(payload)},
                ],
                temperature=1,
            )
            data = json.loads(resp.choices[0].message.content)
            idx = next(iter(data.values()))
            if isinstance(idx, int) and 0 <= idx < n:
                return idx
        except Exception:
            pass
        if attempt < retries:
            time.sleep(backoff + random.random() * 0.3)
            backoff *= 1.6
    # deterministic fallback
    return _det_pick_idx(n, fallback_key)


def _parse_refs(val: object) -> set[str]:
    """Accept NaN or string; split on ; | , and collapse whitespace."""
    if pd.isna(val):
        return set()
    parts = re.split(r"[;|,]+", str(val))
    return {p.strip() for p in parts if p and p.strip()}


def create_mapping_parallel_simple(
    df_items: pd.DataFrame,
    df_accounts: pd.DataFrame,
    df_customers: Optional[pd.DataFrame]=None,
    df_vendors: Optional[pd.DataFrame]=None,
    *,
    df_bu_companies: Optional[pd.DataFrame]=None,   # expects bu_key, sells_to, buys_from
    name_column: str="product_name",
    model: str="gpt-4.1",
    max_workers: int=12
) -> pd.DataFrame:

    # --- validation (as before) ---
    for req in [["name"]]:
        miss = [c for c in req if c not in df_items.columns]
        if miss: raise ValueError(f"df_items missing {miss}")
    miss = [c for c in ["AccountKey","name","GLLevel02"] if c not in df_accounts.columns]
    if miss: raise ValueError(f"df_accounts missing {miss}")

    ACC_ALLOWED = {
        "product_name": "Net Sales",
        "service_name": "Operating Expenses",
        "procurement_name": "Assets",
    }[name_column]

    dotenv.load_dotenv()
    api_key = dotenv.get_key(".env","api_key")
    client = OpenAI(api_key=api_key)

    # --- NEW: build BU relation maps ---
    bu_key_set: set[str] = set()
    sells_map: dict[str, set[str]] = {}
    buys_map:  dict[str, set[str]] = {}
    bu_names_df = None

    if df_bu_companies is not None and not df_bu_companies.empty:
        if "bu_key" not in df_bu_companies.columns:
            raise ValueError("df_bu_companies must include 'bu_key'")
        bu_key_set = set(df_bu_companies["bu_key"].astype(str))
        # allow picking own BU with your shortlist helper (needs a 'name' col)
        bu_names_df = pd.DataFrame({"name": sorted(bu_key_set)})

        # build adjacency with reciprocity checks later
        for _, r in df_bu_companies.iterrows():
            k = str(r["bu_key"])
            sells_map[k] = _parse_refs(r.get("sells_to"))
            buys_map[k]  = _parse_refs(r.get("buys_from"))

    # --- helpers to build candidate pools ---
    def _combined_partner_pool():
        if name_column == "product_name":
            base = df_customers
        else:
            base = df_vendors

        if base is not None and not base.empty:
            if "name" in base.columns:
                base_use = base[["name"]].copy()
            elif "customer_name" in base.columns:
                base_use = base.rename(columns={"customer_name":"name"})[["name"]].copy()
            elif "vendor_name" in base.columns:
                base_use = base.rename(columns={"vendor_name":"name"})[["name"]].copy()
            else:
                base_use = pd.DataFrame(columns=["name"])
        else:
            base_use = pd.DataFrame(columns=["name"])

        if bu_names_df is not None and not bu_names_df.empty:
            return pd.concat([base_use, bu_names_df], ignore_index=True).drop_duplicates("name")
        return base_use

    def _allowed_bu_partners(own_bu: str) -> list[str]:
        """Return BU partner keys allowed by graph + reciprocity."""
        if not own_bu or own_bu not in bu_key_set:
            return []

        if name_column == "product_name":
            # seller → choose from seller.sells_to; require reciprocal buyer.buys_from contains seller
            candidates = sells_map.get(own_bu, set())
            allowed = [p for p in candidates if own_bu in buys_map.get(p, set())]
        else:
            # buyer → choose from buyer.buys_from; require reciprocal seller.sells_to contains buyer
            candidates = buys_map.get(own_bu, set())
            allowed = [p for p in candidates if own_bu in sells_map.get(p, set())]

        # keep only known BUs
        return [p for p in allowed if p in bu_key_set]

    def worker(item_name: str):
        # 1) Pick account (unchanged, but use ACC_ALLOWED)
        acc_cands = _shortlist_accounts(df_accounts, ACC_ALLOWED, item_name, k=80) or \
                    _shortlist_accounts(df_accounts, ACC_ALLOWED, "", k=80)

        acc_payload = {
            "task":"Select one GL account by index",
            "item_category": name_column,
            "item_name": item_name,
            "candidates": acc_cands,
            "output_schema": {"account_idx": 0},
        }
        acc_idx = _pick_idx_with_gpt(
            client,
            'Return STRICT JSON: {"account_idx": <int>}.',
            acc_payload, len(acc_cands),
            f"{item_name}|{name_column}|account",
            model=model
        )
        acct = acc_cands[acc_idx]
        account_name = acct["name"]

        # 2) Pick OWN BU first (as you had)
        own_bu = None
        if bu_names_df is not None and not bu_names_df.empty:
            bu_cands = _shortlist_partners(bu_names_df, item_name, account_name, k=50)
            if bu_cands:
                bu_payload = {
                    "task": ("Select the SELLER BU by index" if name_column=="product_name"
                            else "Select the BUYER BU by index"),
                    "item_name": item_name,
                    "account_name": account_name,
                    "candidates": bu_cands,
                    "output_schema": {"bu_idx": 0},
                }
                bu_idx = _pick_idx_with_gpt(
                    client,
                    'Return STRICT JSON: {"bu_idx": <int>}.',
                    bu_payload, len(bu_cands),
                    f"{item_name}|{acct['name']}|own_bu",
                    model=model
                )
                own_bu = bu_cands[bu_idx]["name"]

        # 3) Build partner pool from BU graph; FALLBACK to combined pool if empty
        if own_bu:
            allowed_partner_keys = _allowed_bu_partners(own_bu)
        else:
            allowed_partner_keys = []

        if allowed_partner_keys:
            partner_pool = pd.DataFrame({"name": allowed_partner_keys})
        else:
            partner_pool = _combined_partner_pool()

        partner_cands = _shortlist_partners(partner_pool, item_name, account_name, k=200)

        # Select up to 3 distinct partners
        rows_for_item = []
        used_names = set()
        picks_needed = 3

        for pick_idx in range(picks_needed):
            # filter out already used partner names
            filtered = [c for c in partner_cands if c["name"] not in used_names]
            if not filtered:
                break

            par_payload = {
                "task":"Select one partner by index",
                "item_name": item_name,
                "account_name": account_name,
                "candidates": filtered,
                "output_schema": {"partner_idx": 0},
            }
            pidx = _pick_idx_with_gpt(
                client,
                'Return STRICT JSON: {"partner_idx": <int>}.',
                par_payload, len(filtered),
                f"{item_name}|{acct['name']}|partner|{pick_idx}",
                model=model
            )
            partner_name = filtered[pidx]["name"]
            used_names.add(partner_name)

            rows_for_item.append({
                "name": item_name,
                "account_id": acct["AccountKey"],
                "account_name": account_name,
                "customer_name": partner_name if name_column=="product_name" else None,
                "vendor_name":   partner_name if name_column!="product_name" else None,
                "bu_key":        own_bu,
            })

        # Ensure at least one row (fallback)
        if not rows_for_item:
            rows_for_item.append({
                "name": item_name,
                "account_id": acct["AccountKey"],
                "account_name": account_name,
                "customer_name": None if name_column!="product_name" else "Unknown Customer",
                "vendor_name":   None if name_column=="product_name" else "Unknown Vendor",
                "bu_key":        own_bu,
            })

        return rows_for_item

    rows = []
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futs = [ex.submit(worker, str(r["name"]).strip()) for _, r in df_items.iterrows()]
        for f in as_completed(futs):
            rows.extend(f.result())

    return pd.DataFrame(
        rows,
        columns=["name","account_id","account_name","customer_name","bu_key", "vendor_name"]
    )


def map_procurement_services(
    df_procurement: pd.DataFrame,
    df_services: pd.DataFrame,
    df_accounts: pd.DataFrame,
    df_customers: pd.DataFrame,
    df_vendors: pd.DataFrame,
    df_bu_companies: pd.DataFrame
    ) -> tuple[pd.DataFrame, pd.DataFrame]:
    
    """
    Combines procurement and service data into a unified spend dataset.

    Returns:
        - df_spend: Combined spend data with item_name, source_type, and *_id columns
        - df_mapping: Combined mapping with item_name and GL/cost center info
    """

    # Create mappings
    procurement_mapping = create_mapping_parallel_simple(
        df_items=df_procurement, df_accounts=df_accounts, df_customers=df_customers, df_vendors=df_vendors, df_bu_companies=df_bu_companies, name_column="procurement_name"
    )
    print("✔ Procurement mapping done!")
    services_mapping = create_mapping_parallel_simple(
        df_items=df_services, df_accounts=df_accounts, df_customers=df_customers, df_vendors=df_vendors, df_bu_companies=df_bu_companies, name_column="service_name"
    )
    print("✔ Service mapping done!")

    # Assign ID and metadata columns
    df_procurement = df_procurement.copy()
    df_procurement["procurement_id"] = df_procurement["name"]
    df_procurement["service_id"] = None
    df_procurement["product_id"] = None
    df_procurement["item_name"] = df_procurement["name"]
    df_procurement["unit_price"] = df_procurement["unit_price"].fillna(0) 
    df_procurement["source_type"] = "procurement"

    df_services = df_services.copy()
    df_services["service_id"] = df_services["name"]
    df_services["procurement_id"] = None
    df_services["product_id"] = None
    df_services["item_name"] = df_services["name"]
    df_services["unit_price"] = df_services["unit_price"].fillna(0)  
    df_services["source_type"] = "service"

    # Combine and select columns
    df_spend = pd.concat([df_procurement, df_services], ignore_index=True)
    df_spend = df_spend[[
        "item_name", "source_type", "annual_spend", "unit_price", "proportionality",
        "product_id", "procurement_id", "service_id"
    ]]

    # Add item_name to mappings for merge compatibility
    procurement_mapping["item_name"] = procurement_mapping["name"]
    services_mapping["item_name"] = services_mapping["name"]

    df_mapping = pd.concat([procurement_mapping, services_mapping], ignore_index=True)
    df_mapping = df_mapping[["item_name", "account_id", "account_name", "vendor_name", "bu_key"]] 
    
    return df_spend, df_mapping

def map_products(
    df_products: pd.DataFrame,
    df_accounts: pd.DataFrame,
    df_customers: pd.DataFrame,
    df_vendors: pd.DataFrame,
    df_bu_companies: pd.DataFrame
    ) -> tuple[pd.DataFrame, pd.DataFrame]:

    # Assign ID and metadata columns
    df_products["product_id"] = df_products["name"]
    df_products["procurement_id"] = None
    df_products["service_id"] = None
    df_products["item_name"] = df_products["name"]
    df_products["source_type"] = "Product Sales"

    # Build spend data
    df_spend = df_products[[
        "item_name", "source_type", "annual_spend", "unit_price", "proportionality",
        "product_id", "procurement_id", "service_id"
    ]]

    # Build mapping
    df_mapping = create_mapping_parallel_simple(
        df_items=df_products,
        df_accounts=df_accounts,
        df_customers=df_customers,
        df_vendors=df_vendors,
        df_bu_companies=df_bu_companies,
        name_column="product_name"
    )

    df_mapping["item_name"] = df_mapping["name"]
    df_mapping = df_mapping[["item_name", "account_id", "account_name", "bu_key", "customer_name"]]
    return df_spend, df_mapping

In [8]:
df_bu_companies

,companyname,companycode,bu_key,bu_name,bu_type,sells_to,buys_from
0,BioCirc_denmark,1000,BIOCIRC_BIOGAS_PRODUCTION_DK,Biogas Production,Manufacturing,BIOCIRC_GREEN_HYDROGEN_DK,NaN
1,BioCirc_denmark,1000,BIOCIRC_GREEN_HYDROGEN_DK,Green Hydrogen Generation,Manufacturing,BIOCIRC_CARBON_CONSULTING_DK,BIOCIRC_BIOGAS_PRODUCTION_DK
2,BioCirc_denmark,1000,BIOCIRC_CARBON_CONSULTING_DK,Carbon Mitigation Consulting,Consulting,NaN,BIOCIRC_GREEN_HYDROGEN_DK
3,BioCirc_sweden,1001,BIOCIRC_BIOMASS_COLLECTION_SE,Biomass Collection & Sorting,Services,BIOCIRC_WASTE_UPCYCLING_SE,BIOCIRC_DISTRICT_HEATING_SE
4,BioCirc_sweden,1001,BIOCIRC_WASTE_UPCYCLING_SE,Waste Upcycling R&D,RnD,NaN,BIOCIRC_BIOMASS_COLLECTION_SE
5,BioCirc_germany,1002,BIOCIRC_BIOFUELS_DE,Biofuels Production,Manufacturing,BIOCIRC_ENERGY_TRADING_DE,NaN
6,BioCirc_germany,1002,BIOCIRC_ENERGY_TRADING_DE,Energy Trading & Optimization,Services,BIOCIRC_DIGITAL_OPERATIONS_DE,BIOCIRC_BIOFUELS_DE
7,BioCirc_germany,1002,BIOCIRC_DIGITAL_OPERATIONS_DE,Digital Plant Operations,SharedServices,NaN,BIOCIRC_ENERGY_TRADING_DE
8,BioCirc_uk,1003,BIOCIRC_RESOURCE_RECOVERY_UK,Resource Recovery Services,Services,BIOCIRC_SUSTAINABILITY_RND_UK,BIOCIRC_CIRCULAR_PACKAGING_UK


In [9]:
# Apply proportional spend estimates
df_procurement["annual_spend"] = np.round(1000000 * df_procurement["proportionality"], -3)
df_products["annual_spend"] = np.round(100000 * df_products["proportionality"], -3)
df_services["annual_spend"] = np.round(100000 * df_services["proportionality"], -3)
df_departments["annual_spend"] = np.round(100000 * df_departments["proportionality"], -3)
print("\n * Semantic mapping started * :")

df_erp_expenses, df_map_expenses = map_procurement_services(df_procurement=df_procurement, df_services=df_services, df_accounts=df_accounts, df_customers=df_customers, df_bu_companies=df_bu_companies, df_vendors=df_vendors)
df_erp_products, df_map_products = map_products(df_products=df_products, df_accounts=df_accounts, df_customers=df_customers, df_bu_companies=df_bu_companies, df_vendors=df_vendors)


 * Semantic mapping started * :
✔ Procurement mapping done!
✔ Service mapping done!


In [11]:
df_map_expenses

,item_name,account_id,account_name,vendor_name,bu_key
0,Organic waste (municipal),1003,Raw Materials,BIOCIRC_BIOMASS_COLLECTION_SE,BIOCIRC_WASTE_UPCYCLING_SE
1,Food industry by-products,1003,Raw Materials,Sustainable Starch Co,BIOCIRC_BIOMASS_COLLECTION_SE
2,Food industry by-products,1003,Raw Materials,LabSystems International,BIOCIRC_BIOMASS_COLLECTION_SE
3,Food industry by-products,1003,Raw Materials,BIOCIRC_BIOFUELS_DE,BIOCIRC_BIOMASS_COLLECTION_SE
4,Energy crops (rye grass),1003,Raw Materials,BioSeed Corp,BIOCIRC_BIOMASS_COLLECTION_SE
...,...,...,...,...,...
355,Annual Parking Permit Fee,5022,Other Expenses,CargoSure Services,BIOCIRC_BIOFUELS_DE
356,Annual Parking Permit Fee,5022,Other Expenses,QuickTrack Express,BIOCIRC_BIOFUELS_DE
357,Annual Water Testing Fee,5006,Utilities,LabSystems International,BIOCIRC_BIOGAS_PRODUCTION_DK
358,Annual Water Testing Fee,5006,Utilities,FlexiLine Conveyors,BIOCIRC_BIOGAS_PRODUCTION_DK


In [ ]:
def remap_vendors_customers_with_bu(
    df_customers: pd.DataFrame,
    df_vendors: pd.DataFrame,
    df_bu_companies: pd.DataFrame
    ) -> tuple[pd.DataFrame, pd.DataFrame]:

    """
    Remaps customers and vendors by adding business units (BUs) as intercompany entities.

    Args:
        df_customers (pd.DataFrame): DataFrame containing customer data.
        df_vendors (pd.DataFrame): DataFrame containing vendor data.
        df_bu_companies (pd.DataFrame): DataFrame containing business unit data with 'bu_key'.

    Returns:
        tuple: Updated DataFrames for customers and vendors with BUs added.
    """

     # Create BU entries for customers and vendors

    df_bu = pd.DataFrame()
    df_bu["name"] = df_bu_companies["bu_key"]
    df_bu["proportionality"] = 1 / len(df_bu_companies)
    df_bu["customer_segment"] = "Intercompany"
    df_bu["vendor_segment"] = "Intercompany"

    start_id_cust = int(df_customers["customer_id"].max()) + 1
    end_id_cust   = start_id_cust + len(df_bu)

    start_id_vendor = int(df_vendors["vendor_id"].max()) + 1
    end_id_vendor   = start_id_vendor + len(df_bu)

    df_bu["customer_id"] = range(start_id_cust, end_id_cust)
    df_bu["vendor_id"] = range(start_id_vendor, end_id_vendor)

    df_bu_companies["customer_id"] = range(start_id_cust, end_id_cust)
    df_bu_companies["vendor_id"] = range(start_id_vendor, end_id_vendor)

    df_customers = pd.concat([df_customers, df_bu.drop(columns=["vendor_id", "vendor_segment"])], axis=0, ignore_index=True)
    df_vendors = pd.concat([df_vendors, df_bu.drop(columns=["customer_id", "customer_segment"])], axis=0, ignore_index=True)

    df_customers = utils.convert_column_to_percentage(df_customers, "proportionality", scale=1.0)
    df_vendors = utils.convert_column_to_percentage(df_vendors, "proportionality", scale=1.0)

    return df_customers, df_vendors, df_bu_companies

In [34]:
df_bu_companies

,companyname,companykey,bu_key,bu_name,bu_type,sells_to,buys_from,customer_id,vendor_id
0,BioCirc_denmark,1000,BIOCIRC_BIOGAS_PLANT,Biogas Production,Manufacturing,BIOCIRC_UPGRADING,NaN,73,79
1,BioCirc_denmark,1000,BIOCIRC_UPGRADING,Biogas Upgrading,Manufacturing,BIOCIRC_GRID_INJECTION,BIOCIRC_BIOGAS_PLANT,74,80
2,BioCirc_denmark,1000,BIOCIRC_CARBON_CONSULTING,Carbon Consulting,Consulting,NaN,NaN,75,81
3,BioCirc_sweden,1001,BIOCIRC_GRID_INJECTION,Gas Grid Injection,Services,BIOCIRC_DISTRICT_HEATING,BIOCIRC_UPGRADING,76,82
4,BioCirc_germany,1002,BIOCIRC_ENERGY_TRADING,Renewable Energy Trading,Services,NaN,BIOCIRC_WASTE_LOGISTICS,77,83
5,BioCirc_germany,1002,BIOCIRC_RND_BIOTECH,Biotech Research & Development,RnD,NaN,NaN,78,84
6,BioCirc_netherlands,1003,BIOCIRC_WASTE_LOGISTICS,Waste Collection & Logistics,Services,BIOCIRC_ENERGY_TRADING,NaN,79,85
7,BioCirc_netherlands,1003,BIOCIRC_FEEDSTOCK_PROCUREMENT,Feedstock Procurement,SharedServices,NaN,NaN,80,86
8,BioCirc_netherlands,1003,BIOCIRC_ANAEROBIC_DIGESTION,Anaerobic Digestion,Manufacturing,NaN,BIOCIRC_FEEDSTOCK_PROCUREMENT,81,87
